In [3]:
using Random
using IntervalArithmetic
using Gurobi
using JuMP
using MIPVerify
using MIPVerify:relu
using MIPVerify:NNetFormat,evaluate_network,evaluate_network_withoutNorm,evaluate_network_multiple,info
using MIPVerify:prep_data_file,get_matrix_params
using MIPVerify:JuMPLinearType
using MAT
using Images
using Printf

In [ ]:
##首先找7*7输入下的反例

In [6]:
struct VerifyInfo
    index::Int
    time::Float64
    adv_found::Bool
    adv_not_found::Bool
    verified::Bool
    verified_res::Int
end

In [ ]:
using Printf

function rounding_error_image77()
    result = VerifyInfo[]
    round_error_class = Int[]
    binary_file_path = "../../../resized_images/resized_mnist_images77_test.bin"
    output_file_path = "./misclassified_indices77.txt"
    image_size = (7, 7)
    num_images = 10000

    # Load images and labels
    images, labels = load_all_binary_images(binary_file_path, image_size, num_images)

    # Get network parameters
    nn = MIPVerify.get_example_network_params("F16MNISTinput_77")

    # Define rounding modes
    rounding_modes = [RoundToZero, RoundDown, RoundUp, RoundNearest]

    # Set to track processed indices
    processed_indices = Set{Int}()

    # Open the output file for writing
    open(output_file_path, "w") do file
        for mode in rounding_modes
            setrounding(BigFloat, mode)
            for index in 1:5500
                # Skip if the index has already been processed
                if index in processed_indices
                    continue
                end

                sample_image = reshape(images[index], (1, 7, 7, 1))
                sample_label = labels[index]
                predicted_output16 = Float16.(sample_image) |> nn
                setprecision(BigFloat, 12)
                predicted_output_bf16 = BigFloat.(sample_image) |> nn

                # Get predicted class indices
                predicted_index16 = argmax(predicted_output16) - 1  # Adjusting for 0-based index
                predicted_index_bf16 = argmax(predicted_output_bf16) - 1


                if predicted_index16 != sample_label
                    continue
                end

                if predicted_index16 != predicted_index_bf16
                    # Print and store the results
                    println("\n Index: $index, BigFloat Prediction: $predicted_index_bf16, Float16 Prediction: $predicted_index16, Label: $sample_label")
                    @printf(file, "%d, %d, %d, %d\n", index, predicted_index_bf16, predicted_index16, sample_label)
                    push!(processed_indices, index)  # Mark index as processed
                end
            end
        end
    end
end


In [ ]:
res=rounding_error_image77()

In [ ]:
##现在寻找784输入下反例。

In [ ]:
function rounding_error_image784()
    misclassified = 0
    result = VerifyInfo[]
    round_error_class=Int[]
    mnist = MIPVerify.read_datasets("MNIST")
    nn = MIPVerify.get_example_network_params("F16MNIST24")
    output_file_path = "./misclassified_indices784.txt"
    # Define rounding modes
    rounding_modes = [RoundToZero, RoundDown, RoundUp, RoundNearest]

    # Set to track processed indices
    processed_indices = Set{Int}()
    # Open the output file for writing
    open(output_file_path, "w") do file
        for mode in rounding_modes
            setrounding(BigFloat, mode)
            for index in 1:5500
                # Skip if the index has already been processed
                if index in processed_indices
                    continue
                end

                sample_image = MIPVerify.get_image(mnist.test.images, index)
                sample_label = MIPVerify.get_label(mnist.test.labels, index)

                # Compute outputs with Float16 precision
                predicted_output16 = Float16.(sample_image) |> nn

                # Compute outputs with BigFloat precision
                setprecision(BigFloat, 12)
                predicted_output_bf16 = BigFloat.(sample_image) |> nn

                # Get predicted class indices
                predicted_index16 = argmax(predicted_output16) - 1  # Adjusting for 0-based index
                predicted_index_bf16 = argmax(predicted_output_bf16) - 1


                if predicted_index16 != sample_label
                    continue
                end

                if predicted_index16 != predicted_index_bf16
                    # Print and store the results
                    println("\n Index: $index, BigFloat Prediction: $predicted_index_bf16, Float16 Prediction: $predicted_index16, Label: $sample_label")
                    @printf(file, "%d, %d, %d, %d\n", index, predicted_index_bf16, predicted_index16, sample_label)
                    push!(processed_indices, index)  # Mark index as processed
                end
            end
        end
    end
end

In [ ]:
res784=rounding_error_image784()

In [7]:
function rounding_error_fashion784()
    misclassified = 0
    result = VerifyInfo[]
    round_error_class=Int[]
    mnist = MIPVerify.read_datasets("FASHIONMNIST")
    nn = MIPVerify.get_example_network_params("F16Fashion24")
    output_file_path = "./fsmisclassified_indices784.txt"
    # Define rounding modes
    rounding_modes = [RoundToZero, RoundDown, RoundUp, RoundNearest]

    # Set to track processed indices
    processed_indices = Set{Int}()
    # Open the output file for writing
    open(output_file_path, "w") do file
        for mode in rounding_modes
            setrounding(BigFloat, mode)
            for index in 1:5500
                # Skip if the index has already been processed
                if index in processed_indices
                    continue
                end

                sample_image = MIPVerify.get_image(mnist.test.images, index)
                sample_label = MIPVerify.get_label(mnist.test.labels, index)

                # Compute outputs with Float16 precision
                predicted_output16 = Float16.(sample_image) |> nn

                # Compute outputs with BigFloat precision
                setprecision(BigFloat, 14)
                predicted_output_bf16 = BigFloat.(sample_image) |> nn

                # Get predicted class indices
                predicted_index16 = argmax(predicted_output16) - 1  # Adjusting for 0-based index
                predicted_index_bf16 = argmax(predicted_output_bf16) - 1


                if predicted_index16 != sample_label
                    continue
                end

                if predicted_index16 != predicted_index_bf16
                    # Print and store the results
                    println("\n Index: $index, BigFloat Prediction: $predicted_index_bf16, Float16 Prediction: $predicted_index16, Label: $sample_label")
                    @printf(file, "%d, %d, %d, %d\n", index, predicted_index_bf16, predicted_index16, sample_label)
                    push!(processed_indices, index)  # Mark index as processed
                end
            end
        end
    end
end

rounding_error_fashion784 (generic function with 1 method)

In [9]:
resfs784 = rounding_error_fashion784()


 Index: 1, BigFloat Prediction: 8, Float16 Prediction: 9, Label: 9

 Index: 55, BigFloat Prediction: 8, Float16 Prediction: 2, Label: 2

 Index: 99, BigFloat Prediction: 8, Float16 Prediction: 4, Label: 4

 Index: 611, BigFloat Prediction: 6, Float16 Prediction: 0, Label: 0

 Index: 690, BigFloat Prediction: 6, Float16 Prediction: 4, Label: 4

 Index: 1065, BigFloat Prediction: 6, Float16 Prediction: 4, Label: 4

 Index: 1360, BigFloat Prediction: 6, Float16 Prediction: 4, Label: 4

 Index: 1495, BigFloat Prediction: 6, Float16 Prediction: 0, Label: 0

 Index: 2071, BigFloat Prediction: 9, Float16 Prediction: 7, Label: 7

 Index: 2192, BigFloat Prediction: 0, Float16 Prediction: 6, Label: 6

 Index: 2230, BigFloat Prediction: 6, Float16 Prediction: 2, Label: 2

 Index: 2405, BigFloat Prediction: 4, Float16 Prediction: 6, Label: 6

 Index: 2474, BigFloat Prediction: 8, Float16 Prediction: 4, Label: 4

 Index: 2492, BigFloat Prediction: 8, Float16 Prediction: 6, Label: 6

 Index: 3445, 